# HardCoRe-NAS Training on Kaggle

In [ ]:
# 1. Clone Repository (Skip LFS files to speed up)
# Thay link dưới đây bằng link repo GitHub của bạn
REPO_URL = "https://github.com/thanquannguyen/HardCoReNAS_pi.git"

# GIT_LFS_SKIP_SMUDGE=1 giúp bỏ qua việc tải file .tar nặng (chỉ cần code để train)
!GIT_LFS_SKIP_SMUDGE=1 git clone $REPO_URL
%cd HardCoReNAS_pi

In [ ]:
# 2. Install Dependencies
!pip install -r Docker/requirements.txt
# Cài đặt numpy<2 để tránh lỗi tương thích với các thư viện cũ
!pip install "numpy<2" timm onnx onnxruntime gdown

In [ ]:
# 3. Prepare Data & Checkpoints
!git lfs install
# !git lfs pull --include="checkpoints/hardcorenas_a.pth" # LFS Quota Exceeded

# --- DOWNLOAD FROM GOOGLE DRIVE ---
# 1. Upload file 'checkpoints/hardcorenas_a.pth' lên Google Drive của bạn.
# 2. Chuột phải -> Share -> General access: Anyone with the link -> Copy link.
# 3. Link sẽ có dạng: https://drive.google.com/file/d/1xYz.../view?usp=sharing
# 4. Copy phần ID (1xYz...) và dán vào biến bên dưới:

GDRIVE_FILE_ID = "1wMbls56vjGgXG-V8C0fRSeg2H9vHSDGp" # <--- DÁN ID FILE VÀO ĐÂY

import os
import gdown

checkpoint_path = "checkpoints/hardcorenas_a.pth"

if GDRIVE_FILE_ID != "YOUR_FILE_ID_HERE":
    print(f"Downloading checkpoint from Google Drive (ID: {GDRIVE_FILE_ID})...")
    # Xóa file pointer cũ nếu có
    if os.path.exists(checkpoint_path):
        os.remove(checkpoint_path)
    
    # Download file mới
    url = f'https://drive.google.com/uc?id={GDRIVE_FILE_ID}'
    gdown.download(url, checkpoint_path, quiet=False)
else:
    print("⚠️ CHƯA NHẬP GDRIVE_FILE_ID! Vui lòng nhập ID file từ Google Drive để tải checkpoint.")
    # Nếu bạn không dùng Drive, hãy đảm bảo file checkpoint đã được upload thủ công vào đúng thư mục

!ls -lh checkpoints/hardcorenas_a.pth

# Tải và chuẩn bị dữ liệu CIFAR-10
!python scripts/convert_cifar10.py

In [ ]:
# 4. Run Training Pipeline
# Script này sẽ thực hiện:
# - Step 1: Measure Latency (CPU)
# - Step 3: Fine-tune (Training 100 epochs trên GPU)
# - Step 4: Compression (Pruning -> Quantization)

# PYTHONPATH=. giúp Python tìm thấy module timm trong thư mục hiện tại thay vì timm cài từ pip
!PYTHONPATH=. python scripts/run_nas_pipeline.py


# !python train.py data/cifar10_images \
#   --mobilenet_string="[['ds_r1_k3_s1_e1_c16_nre'], ['ir_r1_k5_s2_e3_c24_nre', 'ir_r1_k5_s1_e3_c24_nre_se0.25'], ['ir_r1_k5_s2_e3_c40_nre', 'ir_r1_k5_s1_e6_c40_nre_se0.25'], ['ir_r1_k5_s2_e6_c80_se0.25', 'ir_r1_k5_s1_e6_c80_se0.25'], ['ir_r1_k5_s1_e6_c112_se0.25', 'ir_r1_k5_s1_e6_c112_se0.25'], ['ir_r1_k5_s2_e6_c192_se0.25', 'ir_r1_k5_s1_e6_c192_se0.25'], ['cn_r1_k1_s1_c960']]" \
#   --num-classes 10 \
#   --initial-checkpoint checkpoints/hardcorenas_a.pth \
#   --epochs 50 \
#   --batch-size 128 \
#   --workers 4 \
#   --amp \
#   --output outputs/fine_tuned

In [ ]:
# 5. Zip & Download Outputs
# Nén thư mục output để tải về
!zip -r fine_tuned_model.zip outputs/fine_tuned
!zip -r compressed_model.zip compressed_model.pth model_quantized.onnx

print("Done! Bạn có thể tải file .zip ở phần Output bên phải.")